# Generating a Sankey diagram

In [1]:
import numpy as np 
import pandas as pd 

In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Prepare dummy data

In [2]:
#df_query.columns =['token', 'state', 'evnt_ts']

In [3]:
df = pd.DataFrame(columns=["token", "state", "evnt_ts"])

df = df.append({"token": 1, "state":"A",  "evnt_ts":pd.to_datetime("2019-01-01 00:00:00")  }, ignore_index=True)
df = df.append({"token": 1, "state":"B",  "evnt_ts":pd.to_datetime("2019-01-01 00:01:00")  }, ignore_index=True)
df = df.append({"token": 1, "state":"C",  "evnt_ts":pd.to_datetime("2019-01-01 00:02:00")  }, ignore_index=True)
df = df.append({"token": 1, "state":"D",  "evnt_ts":pd.to_datetime("2019-01-01 00:03:00")  }, ignore_index=True)
df = df.append({"token": 1, "state":"E",  "evnt_ts":pd.to_datetime("2019-01-01 00:04:00")  }, ignore_index=True)
df = df.append({"token": 1, "state":"F",  "evnt_ts":pd.to_datetime("2019-01-01 00:05:00")  }, ignore_index=True)

df = df.append({"token": 2, "state":"A",  "evnt_ts":pd.to_datetime("2019-01-01 00:00:00")  }, ignore_index=True)
df = df.append({"token": 2, "state":"C",  "evnt_ts":pd.to_datetime("2019-01-01 00:01:00")  }, ignore_index=True)
df = df.append({"token": 2, "state":"B",  "evnt_ts":pd.to_datetime("2019-01-01 00:02:00")  }, ignore_index=True)
df = df.append({"token": 2, "state":"D",  "evnt_ts":pd.to_datetime("2019-01-01 00:03:00")  }, ignore_index=True)
df = df.append({"token": 2, "state":"E",  "evnt_ts":pd.to_datetime("2019-01-01 00:04:00")  }, ignore_index=True)
df = df.append({"token": 2, "state":"F",  "evnt_ts":pd.to_datetime("2019-01-01 00:05:00")  }, ignore_index=True)

df = df.append({"token": 3, "state":"A",  "evnt_ts":pd.to_datetime("2019-01-01 00:00:00")  }, ignore_index=True)
df = df.append({"token": 3, "state":"F",  "evnt_ts":pd.to_datetime("2019-01-01 00:05:00")  }, ignore_index=True)


df = df.append({"token": 4, "state":"A",  "evnt_ts":pd.to_datetime("2019-01-01 00:00:00")  }, ignore_index=True)
df = df.append({"token": 4, "state":"E",  "evnt_ts":pd.to_datetime("2019-01-01 00:04:00")  }, ignore_index=True)
df = df.append({"token": 4, "state":"F",  "evnt_ts":pd.to_datetime("2019-01-01 00:05:00")  }, ignore_index=True)

df = df.append({"token": 5, "state":"A",  "evnt_ts":pd.to_datetime("2019-01-01 00:00:00")  }, ignore_index=True)
df = df.append({"token": 5, "state":"D",  "evnt_ts":pd.to_datetime("2019-01-01 00:03:00")  }, ignore_index=True)
df = df.append({"token": 5, "state":"E",  "evnt_ts":pd.to_datetime("2019-01-01 00:04:00")  }, ignore_index=True)
df = df.append({"token": 5, "state":"F",  "evnt_ts":pd.to_datetime("2019-01-01 00:05:00")  }, ignore_index=True)

df_query = df

# Post process - add log_start and calculate drop off

In [4]:
###############################################################################

df_temp=pd.DataFrame({'token':df_query.token.values
                      ,'state':df_query.state.values
                      ,'evnt_ts':pd.to_datetime(df_query.evnt_ts.values, utc=True)})

# plot with all the data as is (can have excessive loops)
#df = df_temp

# plot only the first visit to a state
df = pd.DataFrame({'evnt_ts':df_temp.groupby(['token','state'])['evnt_ts'].min()}).reset_index()

In [5]:
# Add 'log_start' to each token series (helps get clean sankey diagrams)

from datetime import timedelta
df_log_start_ts = df.groupby(['token'])['evnt_ts'].min()-timedelta(seconds=1)
df_log_end_ts   = df.groupby(['token'])['evnt_ts'].max()+timedelta(seconds=1)

df_log_start = pd.DataFrame({'token':df_log_start_ts.index.values, 'state':'log_start', 'evnt_ts':df_log_start_ts.values})
df_log_end  = pd.DataFrame({'token':df_log_end_ts.index.values  , 'state':'log_end'  , 'evnt_ts':df_log_end_ts.values  })

df = df.append(df_log_start, ignore_index=True)
#df = df.append(df_log_end, ignore_index=True)
df.evnt_ts = pd.to_datetime(df.evnt_ts, utc=True)

df['forward_rank'] = df.groupby(['token'])['evnt_ts'].rank(ascending=True)
df['forward_rank_plus_one'] = df.groupby(['token'])['evnt_ts'].rank(ascending=True)+1
df['reverse_rank'] = df.groupby(['token'])['evnt_ts'].rank(ascending=False)

df = df.sort_values(by=['token','evnt_ts'])
df.reset_index(inplace=True, drop=True)

In [6]:
###############################################################################
# calculate 'state' metrics like visits, drops and percent_drop

df_state_visits = pd.DataFrame({'visits' : df.groupby(['state'])['token'].nunique()}).reset_index()
df_state_drops  = pd.DataFrame({'drops'  : df[df['reverse_rank']==1].groupby(['state'])['token'].nunique()}).reset_index()
df_state_metrics=pd.merge(df_state_visits, df_state_drops, on='state', how='left')
df_state_metrics['visitors_percent']=np.round(df_state_metrics.visits.values/df.token.nunique()*100,1)
df_state_metrics['drop_off_percent']=np.round(df_state_metrics.drops.values/df.token.nunique()*100,1)
df_state_metrics['stage_drop_off_percent']=np.round(df_state_metrics.drops.values/df_state_metrics.visits.values*100,1)
df_state_metrics = df_state_metrics.fillna(0)

In [7]:
##############################################################################
# calculate state transition metrics 

df_t = pd.merge(df, df,  how='inner', left_on=['token','forward_rank_plus_one'], right_on = ['token','forward_rank'])

df_state_transitions = pd.DataFrame({'counts' : df_t.groupby(['state_x','state_y'])['token'].nunique()}).reset_index()
df_state_transitions['percent']=np.round(df_state_transitions.counts.values / df_t.token.nunique()*100,1)

# Filter small transitions

In [8]:
###############################################################################

# filter off the small transitions
filter_percent_cutoff = 0

df_final = df_state_transitions[df_state_transitions.percent>=filter_percent_cutoff]
df_final.sort_values(by=['percent'], axis=0, ascending=False, inplace=True)
df_final.reset_index(inplace=True, drop=True)

node_list = list(df_state_metrics.state.values)
node_label = [df_state_metrics.state[i] + 
             '(' +  str(df_state_metrics.visitors_percent[i]) + '%V '
                 +  str(df_state_metrics.drop_off_percent[i]) + '%D '  
                 +  str(df_state_metrics.stage_drop_off_percent[i]) + '%SD)'                
             for i in range(df_state_metrics.shape[0]) ]

In [9]:
df_state_metrics

,state,visits,drops,visitors_percent,drop_off_percent,stage_drop_off_percent
0,A,5,0.0,100.0,0.0,0.0
1,B,2,0.0,40.0,0.0,0.0
2,C,2,0.0,40.0,0.0,0.0
3,D,3,0.0,60.0,0.0,0.0
4,E,4,0.0,80.0,0.0,0.0
5,F,5,5.0,100.0,100.0,100.0
6,log_start,5,0.0,100.0,0.0,0.0


In [10]:
# These are the "Tableau 20" colors as RGB.    
# http://www.randalolson.com/2014/06/28/how-to-make-beautiful-data-visualizations-in-python-with-matplotlib/
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.) 
from matplotlib import colors
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
viridis = cm.get_cmap('viridis', 12)
#node_colors=[colors.rgb2hex(viridis(i/len(node_list))[:3]) for i in range(len(node_list))]
node_colors=[colors.rgb2hex(tableau20[i % len(tableau20)]) for i in range(len(node_list))]

# Generate Sankey

In [11]:
import numpy as np
import plotly as ptly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
print (__version__) # requires version >= 1.9.0

#Always run this the command before at the start of notebook
init_notebook_mode(connected=True)

4.14.3


In [12]:

###############################################################################

# populate source & target indexes and value (height of the connection)
df_final['Source']=0
df_final['Target']=0
df_final['Value']=df_final.percent.values
df_final['Color'] = '#262C46'

pd.options.mode.chained_assignment = None  # default='warn'
for i in range(df_final.shape[0]):
    df_final['Source'][i] = node_list.index(df_final.state_x[i])
    df_final['Target'][i] = node_list.index(df_final.state_y[i])


In [16]:
###############################################################################
data_trace = dict(
  type = 'sankey',
  domain = dict(x = [0.1,0.9], y = [0.1,0.9]),
  orientation = "h",
  valueformat = ".3f",
  arrangement = "freeform",
  node = dict(
    pad = 15,
    thickness = 30,
    line = dict(color = "black", width = 1),
    label = node_label,
    color = node_colors
  ),
  link = dict(
    source = df_final ['Source'].dropna(axis = 0, how = 'any'),
    target = df_final ['Target'].dropna(axis = 0, how = 'any'),
    value = df_final ['Value'].dropna(axis = 0, how = 'any')
  )
)

layout = dict(
  title = "User flow",
  height = 1000,
  width = 1400,
  font = dict(size = 10)
 )

fig = dict(data = [data_trace], layout = layout) 
iplot(fig, validate = False) # for Notebooks
#ptly.offline.plot(fig, validate=False)